# Problem

Popular question and answer (qna) site -  stackoverflow (+ their sister sites) allows for download of monthly data dumps from https://archive.org/details/stackexchange.

With this data, can we classify the questions/answers based on

* Conceptual v/s howto question 
* Beginner v/s intermediate v/s hard/trick
* A particular question is associated with another question in terms of the next things to do or perhaps the pre-requisites?
* Predict the next question a user may ask based on this current search

The taxanomy could be a useful layout of the land for a student of the area.

# Schema

The schema for their data is located @ https://ia800500.us.archive.org/22/items/stackexchange/readme.txt.
    
Unfortunately, the data is dumped in an XML format and there is preliminary effort to convert that data into CSV format. We have written a converter (convert2csv.py) for the tables of interest.

The schemas for the tables of interest are shown below.


## Posts
-----------
- Id
- PostTypeId
  - 1: Question
  - 2: Answer
- ParentID (only present if PostTypeId is 2)
- AcceptedAnswerId (only present if PostTypeId is 1)
- CreationDate
- Score
- ViewCount
- Body
- OwnerUserId
- LastEditorUserId
- LastEditorDisplayName="Jeff Atwood"
- LastEditDate="2009-03-05T22:28:34.823"
- LastActivityDate="2009-03-11T12:51:01.480"
- CommunityOwnedDate="2009-03-11T12:51:01.480"
- ClosedDate="2009-03-11T12:51:01.480"
- Title=
- Tags=
- AnswerCount
- CommentCount
- FavoriteCount

## Comments
---------------------------
- Id
- PostId
- Score
- Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
- CreationDate, e.g.:"2008-09-06T08:07:10.730"
- UserId

## Post History
---------------------------
- Id
- PostHistoryTypeId
    - 1: Initial Title - The first title a question is asked with.
    - 2: Initial Body - The first raw body text a post is submitted with.
    - 3: Initial Tags - The first tags a question is asked with.
    - 4: Edit Title - A question's title has been changed.
    - 5: Edit Body - A post's body has been changed, the raw text is stored here as markdown.
    - 6: Edit Tags - A question's tags have been changed.
    - 7: Rollback Title - A question's title has reverted to a previous version.
    - 8: Rollback Body - A post's body has reverted to a previous version - the raw text is stored here.
    - 9: Rollback Tags - A question's tags have reverted to a previous version.
    - 10: Post Closed - A post was voted to be closed.
    - 11: Post Reopened - A post was voted to be reopened.
    - 12: Post Deleted - A post was voted to be removed.
    - 13: Post Undeleted - A post was voted to be restored.
    - 14: Post Locked - A post was locked by a moderator.
    - 15: Post Unlocked - A post was unlocked by a moderator.
    - 16: Community Owned - A post has become community owned.
    - 17: Post Migrated - A post was migrated.
    - 18: Question Merged - A question has had another, deleted question merged into itself.
    - 19: Question Protected - A question was protected by a moderator
    - 20: Question Unprotected - A question was unprotected by a moderator
    - 21: Post Disassociated - An admin removes the OwnerUserId from a post.
    - 22: Question Unmerged - A previously merged question has had its answers and votes restored.
- PostId
- RevisionGUID: At times more than one type of history record can be recorded by a single action.  
- CreationDate: "2009-03-05T22:28:34.823"
- UserId
- UserDisplayName: populated if a user has been removed and no longer referenced by user Id
- Comment: This field will contain the comment made by the user who edited a post
- Text: A raw version of the new value for a given revision. 
- CloseReasonId
    - 1: Exact Duplicate - This question covers exactly the same ground as earlier questions on this topic; its answers may be merged with another identical question.
    - 2: off-topic
    - 3: subjective
    - 4: not a real question
    - 7: too localized
       
       
## Users
---------------------------
 - Id
 - Reputation
 - CreationDate
 - DisplayName
 - EmailHash
 - LastAccessDate
 - WebsiteUrl
 - Location
 - Age
 - AboutMe
 - Views
 - UpVotes
 - DownVotes
       

# Conversion from XML to CSV

Run python convert2csv.py to convert each of the xml files to their CSV equivalents. For columns/attributes which contain textual data, the converter encodes them with base64 encoding so that handling of quotes and special characters (separators) is avoided. 

When the data is read back into the dataframe, the corresponding decode (from base64) needs to happen. The converter also creates a sample file of 100 rows for each xml data dump converted.

In [191]:
#imports
import pandas as pd
import math
import re
import gensim
from gensim import corpora, models
from gensim.models import word2vec
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


SAMPLE_SIZE = 5000

In [223]:
posts = pd.read_csv('data/posts.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna(subset=['Body','Title'])
posts[['Body','Title', 'Tags']].head(2)

,Body,Title,Tags
0,when should i use can when should i use could...,when do i use can or could,<word-choice><tenses><politeness><subjunctive-...
1,doesn t quint mean five what does that h...,where does the quint in quintessential com...,<etymology>


In [227]:
comments = pd.read_csv('data/comments.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna()
comments[['Score','Text']].head(5)

,Score,Text
0,9,i think you need to edit the title of your que...
1,12,it s correct when you re accessing a method of...
2,2,yes i would think in almost any context where...
3,0,would you say it can certainly be acceptable...
4,4,serg would you expect anything less on a ...


In [228]:
posthistory = pd.read_csv('data/posthistory.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna(subset=['Text'])
posthistory[['Text']].head(5)

,Text
0,when could i use can or when can i use could ...
1,when do i could can could
3,doesn t quint mean five what does that hav...
4,where does the quint in quintessential come ...
6,which is the correct use of these two words an...


In [232]:
users = pd.read_csv('data/users.csv.gz',compression='gzip',nrows=SAMPLE_SIZE).dropna(subset=['AboutMe','Location'])
users[['Location','AboutMe']].head(5)

,Location,AboutMe
0,on the server farm,hi i m not really a person i m a background ...
1,corvallis or,developer on the stack overflow team find me...
2,new york ny,developer on the stack overflow team was dubb...
3,raleigh nc,i design stuff for stack exchange also a prof...
4,california,i slip my front end into the back end and the...


## Further cleansing

* Remove (html) tags & carriage returns from the Text field
* Remove stop words (pick up the nltk stop words)
* Use PorterStemmer to stem words

In [196]:
#global
p_stemmer = PorterStemmer()
stop_words = stopwords.words('english')
stop_words.append('use')
#print(stop_words)

In [215]:
class SentenceTokens():
    def __init__(self,df,field):
        self.field = field
        self.df = df
    
    def __iter__(self):
      for index, row in self.df.iterrows():
         raw_sentence = row[self.field]
         raw_tokens = filter(None, re.split("[ ]+",raw_sentence))
         #stem_tokens = [p_stemmer.stem(tok) for tok in raw_tokens]
         yield [tok for tok in raw_tokens if not tok in stop_words and len(tok) > 1 ]

#all posts is a list of (list of tokens). The inner list of tokens is created once for each post
allposts = SentenceTokens(posts,'Title')

#print([p for p in allposts])   
    

In [216]:
#How frequently each term occurs within each document? We construct a document-term matrix.
dictionary = corpora.Dictionary(allposts)
#print(dictionary.token2id) #maps ids to tokens

In [217]:
#bag of words
corpus = [dictionary.doc2bow(text) for text in allposts]
#corpus, is a list of vectors equal to the number of documents. 
#In each document vector is a series of tuples. 
print(corpus[0])
#(term ID, term frequency) pairs

[(0, 1)]


In [218]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=30)

#num_topics: required. An LDA model requires the user to determine how many topics should be generated. 
#id2word: required. The LdaModel class requires our previous dictionary to map ids to strings.
#passes: optional. The number of laps the model will take through corpus.

In [208]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [220]:
@interact(num_topics=5, num_words=6)
def understand(num_topics, num_words):
    return ldamodel.print_topics(num_topics, num_words)
    
#Each generated topic is separated by a comma.
#Within each topic are the three most probable words to appear in that topic.

[(6,
  u'0.039*vs + 0.022*come + 0.019*word + 0.012*correct + 0.012*difference + 0.010*used'),
 (2,
  u'0.019*vs + 0.018*word + 0.018*nouns + 0.012*origin + 0.012*phrase + 0.010*proper'),
 (8,
  u'0.021*correct + 0.017*different + 0.014*english + 0.014*sentences + 0.013*words + 0.011*something'),
 (4,
  u'0.045*one + 0.038*word + 0.014*correct + 0.014*would + 0.012*difference + 0.008*english'),
 (0,
  u'0.043*correct + 0.025*instead + 0.021*pronunciation + 0.020*usage + 0.018*word + 0.014*english')]

In [222]:
pyLDAvis.gensim.prepare(ldamodel,corpus,dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      11.965464        1       1 -0.116885  0.080274
5      11.956726        1       2  0.166722 -0.095681
2      11.051635        1       3 -0.021001 -0.049113
4       9.714886        1       4  0.045404  0.044141
3       9.522060        1       5 -0.047110 -0.044030
6       9.279630        1       6 -0.021744  0.005752
7       9.277806        1       7  0.088213  0.158144
9       9.216112        1       8  0.022499 -0.001063
1       9.119911        1       9 -0.075942 -0.069062
8       8.895770        1      10 -0.040156 -0.029363, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
1720  Default  31.000000        meaning  31.000000  30.0000  30.0000
823   Default  26.000000            one  26.000000  29.0000  29.0000
1836  Default  17.000000        instead  17.000000  28.0000  28.0000
2092  Default  74.000000             vs  74.000000  27.0000  27.0000
429   Default  41.000000       sentence  41.000000  26.0000  26.0000
862   Default  20.000000            say  20.000000  25.0000  25.0000
801   Default  82.000000        correct  82.000000  24.0000  24.0000
184   Default  48.000000           mean  48.000000  23.0000  23.0000
167   Default  24.000000         versus  24.000000  22.0000  22.0000
1681  Default  19.000000  pronunciation  19.000000  21.0000  21.0000
954   Default  16.000000           come  16.000000  20.0000  20.0000
1600  Default  14.000000          nouns  14.000000  19.0000  19.0000
1219  Default  51.000000     difference  51.000000  18.0000  18.0000
2181  Default  21.000000         proper  21.000000  17.0000  17.0000
1898  Default  19.000000         plural  19.000000  16.0000  16.0000
708   Default  32.000000         phrase  32.000000  15.0000  15.0000
97    Default  80.000000           word  80.000000  14.0000  14.0000
2149  Default  26.000000          usage  26.000000  13.0000  13.0000
76    Default  17.000000          would  17.000000  12.0000  12.0000
1311  Default  16.000000      different  16.000000  11.0000  11.0000
2165  Default  19.000000           like  19.000000  10.0000  10.0000
1435  Default  11.000000      sentences  11.000000   9.0000   9.0000
1213  Default  19.000000           verb  19.000000   8.0000   8.0000
1838  Default  14.000000       singular  14.000000   7.0000   7.0000
115   Default  11.000000            end  11.000000   6.0000   6.0000
770   Default  14.000000           name  14.000000   5.0000   5.0000
1827  Default  13.000000      something  13.000000   4.0000   4.0000
716   Default   5.000000            may   5.000000   3.0000   3.0000
1672  Default  11.000000    appropriate  11.000000   2.0000   2.0000
362   Default   7.000000          speak   7.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
1730  Topic10   1.479668        scratch   2.120189   2.0599  -5.8175
2172  Topic10   1.479627          works   2.120189   2.0599  -5.8175
340   Topic10   1.479627      confusion   2.120191   2.0599  -5.8175
1566  Topic10   1.479577           else   2.120193   2.0598  -5.8176
1316  Topic10   1.479390         coming   2.120188   2.0597  -5.8177
475   Topic10   1.442085       indirect   2.121234   2.0337  -5.8432
1931  Topic10   3.591909         accent   5.642832   1.9679  -4.9306
362   Topic10   4.883866          speak   7.804841   1.9508  -4.6234
1435  Topic10   7.021930      sentences  11.303392   1.9435  -4.2603
1472  Topic10   2.184245           fine   3.525295   1.9409  -5.4280
1267  Topic10   2.184214      languages   3.548984   1.9342  -5.4281
488   Topic10   3.592236       meanings   6.375935   1.8458  -4.9305
271   Topic10   4.298089         object   7.766649   1.8279  -4.7511
1311  Topic10   8.238900      different  16.413967   1.7303  -4.1005
1827  Topic10   5.707361      something  13.519610   1.5572  -4.46

In [211]:
model = word2vec.Word2Vec(allposts, size=200)
model.most_similar(positive = ['correct'])

[('none', 0.26615095138549805),
 ('better', 0.25428104400634766),
 ('like', 0.23603419959545135),
 ('find', 0.22828680276870728),
 ('adjective', 0.22783225774765015),
 ('words', 0.22431403398513794),
 ('list', 0.22357627749443054),
 ('take', 0.210282564163208),
 ('nouns', 0.20473437011241913),
 ('english', 0.20281971991062164)]

## References

* https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
* LDA Viz - http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf
* This dashboard @ https://github.com/dhruvaray/soml
    